In [142]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer/sample_submission.csv
/kaggle/input/customer/Train.csv
/kaggle/input/customer/Test.csv


In [143]:
df_data = pd.read_csv("../input/customer/Train.csv")
df_data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [144]:
df_data = df_data.rename(columns ={'Var_1': 'Category'})
df_data.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [145]:
df_data = df_data.drop(columns=['ID'])
df_data.sample(10)             

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
2283,Female,NaN,29,Yes,Entertainment,0.0,High,1.0,Cat_1,A
795,Male,Yes,29,No,Doctor,NaN,High,2.0,Cat_3,B
2685,Male,Yes,75,Yes,Lawyer,1.0,Low,1.0,Cat_6,A
1477,Female,Yes,68,Yes,Lawyer,1.0,High,2.0,Cat_6,C
5734,Male,No,21,No,Healthcare,NaN,Low,4.0,Cat_3,D
750,Female,No,26,No,Healthcare,10.0,Low,1.0,Cat_6,D
4918,Male,Yes,49,Yes,Artist,1.0,Average,4.0,Cat_6,B
1680,Male,Yes,65,Yes,Executive,0.0,High,4.0,Cat_6,C
2894,Female,Yes,49,Yes,Engineer,6.0,Low,3.0,Cat_3,D
2075,Male,Yes,53,Yes,Artist,3.0,Average,3.0,Cat_6,B


In [146]:
df_data.isna().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Category            76
Segmentation         0
dtype: int64

In [147]:
df_data.dropna().sum()
df_data = df_data.dropna()
df_data.sample(10)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
2038,Female,Yes,53,No,Artist,1.0,High,4.0,Cat_6,A
5904,Female,Yes,60,Yes,Artist,0.0,High,2.0,Cat_6,C
6739,Male,No,25,No,Entertainment,8.0,Low,2.0,Cat_4,D
6561,Male,No,26,Yes,Doctor,1.0,Low,4.0,Cat_6,C
3622,Male,Yes,56,Yes,Entertainment,2.0,Average,2.0,Cat_6,D
1353,Female,Yes,38,Yes,Artist,6.0,Low,4.0,Cat_6,C
3180,Female,No,85,Yes,Lawyer,0.0,Low,3.0,Cat_6,A
6324,Male,Yes,66,No,Lawyer,0.0,High,2.0,Cat_6,B
3520,Male,Yes,58,Yes,Executive,0.0,Average,4.0,Cat_6,C
5501,Male,No,22,No,Healthcare,0.0,Low,5.0,Cat_6,D


# Covert non-numerical values into numerical vaues

In [148]:
def get_encoded_dict(values):
    return {val:i for i, val in enumerate(values)}

In [149]:
gender_dict = get_encoded_dict(df_data.Gender.unique())
married_dict = get_encoded_dict(df_data.Ever_Married.unique())
grad_dict = get_encoded_dict(df_data.Graduated.unique())
Profession_dict = get_encoded_dict(df_data.Profession.unique())
spend_dict = get_encoded_dict(df_data.Spending_Score.unique())
cat_dict = get_encoded_dict(df_data.Category.unique())
segment_dict = get_encoded_dict(df_data.Segmentation.unique())
segment_dict

{'D': 0, 'B': 1, 'C': 2, 'A': 3}

In [150]:
encoded_dict = {
    'Gender' : gender_dict,
    'Ever_Married': married_dict,
    'Graduated' : grad_dict,
    'Profession': Profession_dict,
    'Spending_Score': spend_dict,
     'Category': cat_dict,
    'Segmentation': segment_dict
}

In [151]:
df_data = df_data.replace(encoded_dict)
df_data.sample(10)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Category,Segmentation
6410,0,1,72,1,8,0.0,1,2.0,1,2
3212,0,0,26,1,0,1.0,0,5.0,1,0
3925,1,1,55,1,3,0.0,2,4.0,1,2
1159,0,0,18,0,0,3.0,0,5.0,1,0
560,0,1,43,0,3,1.0,0,2.0,1,1
2968,1,1,40,1,1,1.0,2,4.0,6,1
6060,0,1,38,1,0,8.0,2,4.0,1,2
7791,1,1,38,1,1,9.0,2,3.0,3,2
6487,0,1,38,1,8,0.0,2,3.0,1,1
4386,1,1,81,0,2,0.0,1,2.0,1,1


In [152]:
df_data.columns

Index(['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Category',
       'Segmentation'],
      dtype='object')

In [153]:
feature_cols = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession','Work_Experience', 'Spending_Score', 'Family_Size', 'Category']
target_col = ['Segmentation']

In [154]:
from sklearn.model_selection import train_test_split
data_x = df_data.loc[:, feature_cols]
data_y = df_data.loc[:, target_col]
train_x,test_x,train_y,test_y = train_test_split(data_x, data_y, test_size=0.2)

In [155]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(5332, 9)
(5332, 1)
(1333, 9)
(1333, 1)


In [156]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(train_x.to_numpy(), train_y.to_numpy().reshape(-1))

CategoricalNB()

In [157]:
actual_y = test_y.to_numpy().reshape(-1)
predicted_y = cnb.predict(test_x.to_numpy())

In [158]:
actual_y

array([3, 3, 3, ..., 3, 0, 2])

In [159]:
predicted_y

array([3, 2, 2, ..., 1, 1, 0])

In [161]:
from sklearn import metrics
metrics.accuracy_score(actual_y, predicted_y)

0.5011252813203301

# Confustion matrix,Classification_Report

In [165]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(actual_y,predicted_y))

[[214  21  16  75]
 [ 42  74 139  78]
 [ 48  37 232  20]
 [ 57  68  64 148]]


In [164]:
segment_dict

{'D': 0, 'B': 1, 'C': 2, 'A': 3}

In [166]:
print(classification_report(actual_y, predicted_y, target_names=['D','B','C','A']))

              precision    recall  f1-score   support

           D       0.59      0.66      0.62       326
           B       0.37      0.22      0.28       333
           C       0.51      0.69      0.59       337
           A       0.46      0.44      0.45       337

    accuracy                           0.50      1333
   macro avg       0.48      0.50      0.48      1333
weighted avg       0.48      0.50      0.48      1333

